In [73]:
#Text for saving the notebook
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix, vstack, hstack
from sklearn.metrics import accuracy_score

In [84]:
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)

In [3]:
# count_vect = CountVectorizer()

In [4]:
# count_vect

In [5]:
# X_train_counts = count_vect.fit_transform(twenty_train.data)
# X_train_counts.shape

In [85]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(twenty_train.data)
X_train_tfidf.shape

(11314, 130107)

In [86]:
clf = MultinomialNB(alpha = 0.01).fit(X_train_tfidf, twenty_train.target)

In [87]:
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)
# X_test_counts = count_vect.fit_transform(twenty_test.data)
X_test_tfidf = tfidf_vectorizer.transform(twenty_test.data)
print(X_test_tfidf.shape)
predicted = clf.predict(X_test_tfidf)
predicted

# predicted = text_clf.predict(twenty_test.data)
np.sum(predicted == twenty_test.target)/len(predicted)

(7532, 130107)


0.8352363250132767

# SECOND PART STARTS FROM HERE

In [ ]:
#Step-1
#Splitting the train data into labelled and unlabelled
# labelled_X, unlabelled_X, labelled_y, _ = train_test_split(twenty_train.data[:4000], twenty_train.target[:4000], train_size = 0.2)

# clf.class_log_prior_.shape, clf.feature_log_prob_.shape

In [ ]:
#Finding the TF-IDF from the whole data for labelled and unlabelled data (Feature extraction step)
# tfidf_vectorizer = TfidfVectorizer()
# tfidf_vectorizer_obj = tfidf_vectorizer.fit(twenty_train.data)
# labelled_X_tfidf = tfidf_vectorizer.transform(labelled_X)
# print(labelled_X_tfidf.shape)
# unlabelled_X_tfidf = tfidf_vectorizer.transform(unlabelled_X)
# print(unlabelled_X_tfidf.shape)

In [ ]:
# def calculate_log_likelihood(class_log_prob, feature_log_prob):

In [ ]:

# X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# X_label.shape

In [78]:
#Actual Initialization step where we are fitting a classifier on ONLY labelled data
def perform_EM(labelled_X_tfidf, labelled_y, unlabelled_X_tfidf):
    multinomial_clf = MultinomialNB(alpha = 0.01) #Defacto ALWAYS use this
    multinomial_clf.fit(labelled_X_tfidf, labelled_y)
    class_prob = multinomial_clf.class_log_prior_
    feature_prob = multinomial_clf.feature_log_prob_
    old_loss = 0
    loss = -1*(calculate_unlabeled_loss(class_prob, feature_prob, unlabelled_X_tfidf) + calculate_labelled_loss(class_prob, feature_prob, labelled_X_tfidf, labelled_y))
    diff = (loss - old_loss)
    print("diff before",diff)
    threshold = np.exp(-10)
    while(abs(diff) > threshold):
        old_loss = loss
        #E-step
        pred_labels = multinomial_clf.predict(unlabelled_X_tfidf)

        #Combining the data
        X_total = vstack([labelled_X_tfidf, unlabelled_X_tfidf])
        Y_total = np.concatenate((labelled_y, pred_labels), axis = 0)
        X_total.shape, Y_total.shape


        #M-step
        multinomial_clf.fit(X_total, Y_total)
        
        class_prob = multinomial_clf.class_log_prior_
        feature_prob = multinomial_clf.feature_log_prob_
        print(type(feature_prob), feature_prob.shape)
        loss = -1*( calculate_unlabeled_loss(class_prob, feature_prob, unlabelled_X_tfidf) + calculate_labelled_loss(class_prob, feature_prob, labelled_X_tfidf, labelled_y))

        diff = old_loss - loss
        print("diff", diff)
    return multinomial_clf

In [ ]:
#After the EM step calculate the loss and converge based on it.
#Once that is done then use the trained classifier and predict for the test samples and caculate the accuracy

In [ ]:
# for x in X_total[:1]:
#     print(x)
# X_total[:1].shape

In [ ]:
#Testing the word-document one hot representation:
small_documents = twenty_train.data[:100]
# print(small_documents[:2])
vectorizer = CountVectorizer(binary = True)
doc_term_csr_matrix = vectorizer.fit_transform(small_documents)
#This is a numpy ndarray now
doc_term_numpy_matrix = doc_term_csr_matrix.toarray()
# print(vectorizer.get_feature_names())

In [ ]:
doc_term_numpy_matrix.shape

In [ ]:
np.savetxt("foo.csv", mat.toarray(), delimiter=",", fmt = "%d")
# X_label[:10].toarray()

In [ ]:
vectorizer1 = CountVectorizer(binary=True)
cv_obj = vectorizer1.fit(twenty_train.data)

In [ ]:
mat = cv_obj.transform(small_documents)
# type(mat)
mat.shape

In [ ]:
len(tfidf_vectorizer.get_feature_names())

In [92]:
def calculate_unlabeled_loss(class_prob, feature_prob, unlabelled_data_tfidf):
    """
    unlabeled_data_one_hot_matrix: Assuming this to be ndarray
    """
    print("unlabelled_data shape", unlabelled_data_tfidf.shape)
    n = unlabelled_data_tfidf.shape[0]
    
    p_xi_cj = np.sum(feature_prob, axis = 1, keepdims = True)
    class_prob = class_prob.reshape(class_prob.shape[0],1)
    print(class_prob.shape)
    sum_loss = class_prob + p_xi_cj
    sum_for_all_classes = np.sum(sum_loss, axis = 0, keepdims = True)
    sum_for_all_classes = sum_for_all_classes[0]
    total_loss = n*sum_for_all_classes
    return total_loss

In [80]:
def calculate_labelled_loss(class_prob, feature_prob, labelled_data_tfidf, labelled_labels):
    n_labelled = labelled_data_tfidf.shape[0]
    loss = 0
    sum_of_feature_log_probs = np.sum(feature_prob, axis = 1, keepdims = True)
    for i in range(n_labelled):
        c_index = labelled_labels[i]
        prob_for_c = class_prob[c_index]
        feature_value_for_c = sum_of_feature_log_probs[c_index]
        loss += (prob_for_c + feature_value_for_c)
    return loss

In [27]:
# def calculate_unlabeled_loss(class_prob, feature_prob, unlabeled_data_one_hot_matrix):
#     """
#     unlabeled_data_one_hot_matrix: Assuming this to be ndarray
#     """
#     class_doc_matrix = np.matmul(feature_prob, unlabeled_data_one_hot_matrix.T)
#     print("class_doc_matrix ", class_doc_matrix.shape)
#     #this is the sum loss matrix for all documents and all classes
    
#     class_prob = class_prob.reshape(class_prob.shape[0],1)
#     print(class_prob.shape)
#     sum_loss = class_prob + class_doc_matrix
#     sum_for_all_classes = np.sum(sum_loss, axis = 0, keepdims = True)
#     total_loss = np.sum(sum_for_all_classes, axis = 1)
#     total_loss = total_loss[0]
#     print("unlabeled loss", total_loss)
#     return total_loss

In [25]:
# def calculate_labelled_loss(class_prob, feature_prob, labeled_data_one_hot_matrix, labelled_labels):
#     n_labelled = labeled_data_one_hot_matrix.shape[0]
#     loss = 0
#     for i in range(n_labelled):
#         c_index = labelled_labels[i]
#         prob_for_c = class_prob[c_index]
#         feature_vector_for_c = feature_prob[c_index]
#         word_doc_belonging_vector = labeled_data_one_hot_matrix[c_index]
#         log_p_x_i_c_j = np.matmul(feature_vector_for_c, word_doc_belonging_vector)
#         loss += (prob_for_c + log_p_x_i_c_j)
#     print("labeled loss", loss)
#     return loss

In [ ]:
#Using the new classifier generated from the maximization step
feat = clf_new.feature_log_prob_
print(feat.shape)
class_prob = clf_new.class_log_prior_
unlabeled_data_one_hot_matrix = cv_obj.transform(unlabelled_X)
unlabeled_data_one_hot_matrix = unlabeled_data_one_hot_matrix.toarray()
prod = np.matmul(feat, unlabeled_data_one_hot_matrix.T)

In [ ]:
# type(class_prob), type(unlabeled_data_one_hot_matrix)

In [ ]:
# class_prob.reshape(class_prob.shape[0], 1).shape, prod.shape

In [ ]:
feat[0].shape

In [ ]:
unlabeled_data_one_hot_matrix.shape

In [81]:
#Calling EM and testing the prediction
#These steps needs to be done if pasting in a function or main in .py file
#Step - 1
labelled_X, unlabelled_X, labelled_y, _ = train_test_split(twenty_train.data, twenty_train.target, train_size = 0.2)
# vectorizer1 = CountVectorizer(binary=True)
# cv_obj = vectorizer1.fit(twenty_train.data)
# Finding the TF-IDF from the whole data for labelled and unlabelled data (Feature extraction step)
#Step-2
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(twenty_train.data)
labelled_X_tfidf = tfidf_vectorizer.transform(labelled_X)
print(labelled_X_tfidf.shape)
unlabelled_X_tfidf = tfidf_vectorizer.transform(unlabelled_X)
print(unlabelled_X_tfidf.shape)


(2262, 130107)
(9052, 130107)


In [93]:
em_trained_clf = perform_EM(labelled_X_tfidf, labelled_y, unlabelled_X_tfidf)

unlabelled_data shape (9052, 130107)
(20, 1)
diff before [2.92128488e+11]
<class 'numpy.ndarray'> (20, 130107)
unlabelled_data shape (9052, 130107)
(20, 1)
diff [-1.94134149e+10]
<class 'numpy.ndarray'> (20, 130107)
unlabelled_data shape (9052, 130107)
(20, 1)
diff [-20081958.41949463]
<class 'numpy.ndarray'> (20, 130107)
unlabelled_data shape (9052, 130107)
(20, 1)
diff [-2199613.47625732]
<class 'numpy.ndarray'> (20, 130107)
unlabelled_data shape (9052, 130107)
(20, 1)
diff [0.]


In [94]:
#TESTING
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)
X_test_tfidf = tfidf_vectorizer_obj.transform(twenty_test.data)
predicted = em_trained_clf.predict(X_test_tfidf)

# print(accuracy_score(twenty_test.target, predicted))
np.sum(predicted == twenty_test.target)/len(predicted)

0.7798725438130643

In [ ]:
from nltk.corpus import reuters




In [111]:


cats = reuters.categories()

In [112]:
total_docs = len(reuters.paras())

In [ ]:
total_multi = 0
for c in cats:
    lcat = len(reuters.paras(categories=[c]))
    total_multi += lcat